In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
dataset = pd.read_csv('dataset_loker_downsynd.csv')

print("Dataset Overview:")
print(dataset.head())

Dataset Overview:
      Nama Pekerjaan                Perusahaan      Lokasi          Gaji  \
0      Asisten Dapur         PT Roti Nusantara     Jakarta  Rp 2.500.000   
1      Petugas Kebun        Taman Kota Mandiri     Bandung  Rp 2.000.000   
2  Operator Komputer           PT Digital Jaya    Surabaya  Rp 3.000.000   
3  Pembersih Ruangan       PT Bersih Sejahtera  Yogyakarta  Rp 2.200.000   
4  Fotografer Pemula  Studio Kreatif Nusantara    Denpasar  Rp 3.500.000   

                                 Deskripsi Pekerjaan       Kategori Minat  \
0  Membantu koki menyiapkan bahan masakan dan men...              Memasak   
1  Merawat tanaman, menyiram bunga, dan menjaga k...             Berkebun   
2  Mengoperasikan komputer untuk tugas-tugas sede...      Teknologi dasar   
3  Membersihkan ruangan kerja dan menjaga kebersi...  Lingkungan dan alam   
4  Mengambil foto sederhana dan membantu pengedit...      Fotografi, Seni   

                                Kemampuan Dibutuhkan  \
0  Men

In [15]:
dataset['Kategori Minat'] = dataset['Kategori Minat'].fillna('')
dataset['Kemampuan Dibutuhkan'] = dataset['Kemampuan Dibutuhkan'].fillna('')
dataset['Kondisi Kesehatan'] = dataset['Kondisi Kesehatan'].fillna('')

dataset['Kategori Minat'] = dataset['Kategori Minat'].str.split(', ')
dataset['Kemampuan Dibutuhkan'] = dataset['Kemampuan Dibutuhkan'].str.split(', ')
dataset['Kondisi Kesehatan'] = dataset['Kondisi Kesehatan'].str.split(', ')

In [16]:
minat_encoder = MultiLabelBinarizer()
skills_encoder = MultiLabelBinarizer()
kondisi_encoder = MultiLabelBinarizer()

minat_encoded = minat_encoder.fit_transform(dataset['Kategori Minat'])
skills_encoded = skills_encoder.fit_transform(dataset['Kemampuan Dibutuhkan'])
kondisi_encoded = kondisi_encoder.fit_transform(dataset['Kondisi Kesehatan'])

In [19]:
job_features = np.hstack([
    minat_encoded * 0.6,
    skills_encoded * 0.3,
    kondisi_encoded * 0.1
])

In [20]:
X_train, X_test = train_test_split(job_features, test_size=0.2, random_state=42)

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='sigmoid')  # Output layer
])

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, X_train, epochs=215, batch_size=32, validation_split=0.1)

Epoch 1/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - accuracy: 0.5484 - loss: 0.1015 - val_accuracy: 0.6667 - val_loss: 0.0985
Epoch 2/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6202 - loss: 0.1006 - val_accuracy: 0.5000 - val_loss: 0.0980
Epoch 3/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4972 - loss: 0.1020 - val_accuracy: 0.5000 - val_loss: 0.0977
Epoch 4/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5472 - loss: 0.1004 - val_accuracy: 0.8333 - val_loss: 0.0973
Epoch 5/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5915 - loss: 0.1005 - val_accuracy: 0.8333 - val_loss: 0.0975
Epoch 6/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6780 - loss: 0.0987 - val_accuracy: 0.8333 - val_loss: 0.0973
Epoch 7/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5746 - loss: 0.1002 - val_accuracy: 0.7778 - val_loss: 0.0971
Epoch 8/215
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6349 - loss: 0.0999 - val_accuracy: 0.5000 - val_loss

In [26]:
def rekomendasi_jobspark(minat_user, dataset, minat_encoder, skills_encoder, kondisi_encoder, model, kemampuan_user, kondisi_user, top_n=5):
    user_minat_vector = minat_encoder.transform([minat_user])

    user_skills_vector = np.zeros((user_minat_vector.shape[0], skills_encoded.shape[1]))
    user_conditions_vector = np.zeros((user_minat_vector.shape[0], kondisi_encoded.shape[1]))

    user_vector = np.hstack([
        user_minat_vector * 0.6,
        user_skills_vector * 0.3,
        user_conditions_vector * 0.1
    ])

    user_prediction = model.predict(user_vector.reshape(1, -1))

    nilai_similaritas = cosine_similarity(user_prediction, job_features)

    similiaritas_indeks = nilai_similaritas[0].argsort()[-top_n-1:-1][::-1]

    print("Top Recommendations:")
    for i in similiaritas_indeks:
        rekomendasi_jobspark = dataset.iloc[i]
        print(f"Nama Pekerjaan: {rekomendasi_jobspark['Nama Pekerjaan']}")
        print(f"Perusahaan: {rekomendasi_jobspark['Perusahaan']}")
        print(f"Lokasi: {rekomendasi_jobspark['Lokasi']}")
        print(f"Kualifikasi: {rekomendasi_jobspark['Kualifikasi']}")
        print(f"Minimal Pengalaman: {rekomendasi_jobspark['Minimal Pengalaman']}")
        print(f"Kategori Minat: {rekomendasi_jobspark['Kategori Minat']}")
        print(f"Kemampuan Dibutuhkan: {rekomendasi_jobspark['Kemampuan Dibutuhkan']}")
        print(f"Kondisi Kesehatan: {rekomendasi_jobspark['Kondisi Kesehatan']}")
        print("-" * 40)

In [34]:
minat_user = ['Berkebun']
kemampuan_user = ['Menggunakan alat sederhana']
kondisi_user = ['Tidak cocok di suhu panas']

rekomendasi_jobspark(minat_user, dataset, minat_encoder, skills_encoder, kondisi_encoder, model, kemampuan_user, kondisi_user)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Top Recommendations:
Nama Pekerjaan: Pengelola Kebun Herbal
Perusahaan: Kebun Herbal Nusantara
Lokasi: Solo
Kualifikasi: Lulusan SMA/SMK
Minimal Pengalaman: 1-2 tahun
Kategori Minat: ['Berkebun', 'Lingkungan dan alam']
Kemampuan Dibutuhkan: ['Menggunakan alat sederhana', 'Ketelitian', 'Kesabaran']
Kondisi Kesehatan: ['Tidak cocok di suhu panas', 'Membutuhkan lingkungan kerja tenang']
----------------------------------------
Nama Pekerjaan: Tukang Kebun
Perusahaan: PT Hijau Nusantara
Lokasi: Jakarta
Kualifikasi: Lulusan SMP/SMK
Minimal Pengalaman: 0-1 tahun
Kategori Minat: ['Berkebun', 'Lingkungan dan alam']
Kemampuan Dibutuhkan: ['Menggunakan alat sederhana', 'Ketelitian', 'Kesabaran']
Kondisi Kesehatan: ['Tidak cocok di suhu panas']
----------------------------------------
Nama Pekerjaan: Penjaga Taman
Perusahaan: Taman Hijau Nusantara
Lokasi: Jakarta
Kualifikasi: Lulusan SMP/SMK
Minimal Pengalaman: 0-1 tahun
Kategori Minat: ['Berkebun', 'Lingkung

In [30]:
model.save('model_sisrek_jobspark.h5')